In [ ]:
import cv2
import os
from random import shuffle
from sklearn import svm
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np

sift = cv2.SIFT()

#dot[0] - x; dot[1] - y;
#box[0][0] - x1; box[0][1] - x2
#box[1][0] - x2; box[1][1] - y2
def concat(descs, kp):
    return np.concatenate((descs[100:], np.asarray((kp.pt[0], kp.pt[1], kp.size, kp.angle), dtype=np.float32)))
#     return np.concatenate(np.asarray(descs[k], dtype=np.float32), np.asarray((kp.pt[0], kp.pt[1], kp.size), dtype=np.float32))
                    
def dot_in_box(box, dot):
    if dot[0] >= box[0][0] - 5 and dot[0] <= box[1][0] + 5 and dot[1] >= box[0][1] - 5 and dot[1] <= box[1][1] + 5:
        return True
    else:
        return False

def get_bbox(value):
    bbox = []
    while True:
        pos_a = value.find('(')
        if pos_a == -1:
            return bbox
        pos_b = value.find(')')
        if pos_b == -1:
            return bbox
        substr = value[pos_a + 1:pos_b]
        box = [int(i) for i in substr.split(',')]
        bbox.append([(box[1], box[0]),(box[3], box[2])])
        if pos_b + 1 >= len(value):
            return bbox
        value = value[pos_b + 1:]

#get data
annotations_path = "/home/hotoru/datasets/cvpr15/annotations/planes/"
data_path = "/home/hotoru/datasets/cvpr15/videos/planes/"

annotations = os.listdir(annotations_path)
data = os.listdir(data_path)

annotations.sort()
data.sort()

for i in range(0, len(annotations)):
    annotations[i] = annotations_path + annotations[i]

for i in range(0, len(data)):
    data[i] = data_path + data[i]


#extract features from files

dataset = []
# for file in range(0, len(data)):
for file in range(2, 4):
    annotation_file = open(annotations[file], 'r').readlines()
    vCap = cv2.VideoCapture(data[file])

    for i in range(0, len(annotation_file)):
        bbox = get_bbox(annotation_file[i])
        ret = vCap.set(1, i)
        (ret, frame) = vCap.read()
        if ret == False: break
        if len(bbox) > 0:
            (kp, descs) = sift.detectAndCompute(frame, None)
            features = []
            pos_elem = 0
            neg_elem = 0
            for k in range(0, len(kp)):
                dot_in = False
                for box in bbox:
                    if dot_in_box(box, kp[k].pt):
                        dot_in = True
                if dot_in:
                    features.append([concat(descs[k], kp[k]), 1])
                    pos_elem = pos_elem + 1
                else:
                    features.append([concat(descs[k], kp[k]), 0])
                    neg_elem = neg_elem + 1
                    
            features = sorted(features, key = lambda x: x[1])
#             print("pos: " + str(pos_elem) + " neg: " + str(neg_elem))
            neg_len = neg_elem
            if int(pos_elem * 0.5) < neg_elem:
                neg_len = int(pos_elem * 3)
            dataset = dataset + features[0:neg_len] + features[-pos_elem:]
            
            for box in bbox:
                cv2.rectangle(frame, box[0], box[1], (20, 220, 20), 2)
            frame = cv2.drawKeypoints(frame, kp, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        if i % 10 == 0:
            print(str(file) + " " + str(i))
#         cv2.imshow("vid", frame)
#         if cv2.waitKey(25) & 0xFF == ord('q'):
#             break
            
print("fin")

2 0
2 10
2 20
2 30
2 40
2 50
2 60
2 70


In [65]:
#create dataset
print("len dataset: " + str(len(dataset)))
shuffle(dataset)
pos_elems = 0
neg_elems = 0
X = []
Y = []
for elem in dataset:
    X.append(elem[0])
    Y.append(elem[1])
    if elem[1] == 1:
        pos_elems = pos_elems + 1
    else:
        neg_elems = neg_elems + 1
        
print("pos: " + str(pos_elems) + " neg: " + str(neg_elems))



X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

X_train_normalized = preprocessing.normalize(X_train, norm='l2')
X_test_normalized = preprocessing.normalize(X_test, norm='l2')


# min_max_scaler = preprocessing.MinMaxScaler()
# X_train_minmax = min_max_scaler.fit_transform(X_train)
# X_test_minmax = min_max_scaler.fit_transform(X_test)

len dataset: 22121
pos: 5358 neg: 16763


In [66]:
X_train_normalized[0]


array([ 0.00815973,  0.00652779,  0.00163195,  0.00163195,  0.        ,
        0.        ,  0.        ,  0.        ,  0.14034739,  0.01795141,
        0.00489584,  0.00815973,  0.00163195,  0.        ,  0.        ,
        0.00163195,  0.35086847,  0.01958336,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.01142362,  0.11913209,
        0.00163195,  0.        ,  0.        ,  0.00652779,  0.00652779,
        0.00163195,  0.00815973,  0.00163195,  0.00489584,  0.00652779,
        0.00326389,  0.00163195,  0.        ,  0.        ,  0.        ,
        0.182778  ,  0.01305557,  0.00326389,  0.00815973,  0.00652779,
        0.00163195,  0.        ,  0.00326389,  0.35086847,  0.01305557,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.01142362,  0.16482659,  0.00489584,  0.        ,  0.00163195,
        0.00979168,  0.00326389,  0.        ,  0.00163195,  0.00163195,
        0.00979168,  0.00815973,  0.00489584,  0.00163195,  0.  

In [ ]:
print("training svm")

# param_grid = {'C': [0.01, 0.1, 1, 10, 100], 'max_iter': [1000, 10000]}
# clf = GridSearchCV(svm.LinearSVC(class_weight='balanced'), param_grid)
# clf = clf.fit(X_train_normalized, y_train)
# print("Best estimator found by grid search:")
# print(clf.best_estimator_)

clf = svm.SVC()
clf.fit(X_train_normalized, y_train)
print("testing")
val = clf.score(X_test_normalized, y_test)
print("res " + str(val))
# ytest = clf.predict(X_test)
# accuracy = 0
# for i in range(0, len(y_test)):
#     if ytest[i] == y_test[i]:
#         accuracy = accuracy + 1
# #     print(str(ytest[i]) + " " + str(y_test[i]))

# print(str(accuracy / (1.0*len(y_test))))

training svm


In [71]:
#testing
annotation_file = open(annotations[1], 'r').readlines()
vCap = cv2.VideoCapture(data[1])

for i in range(0, len(annotation_file) - 10):
    bbox = get_bbox(annotation_file[i])
    ret = vCap.set(1, i)
    (ret, frame) = vCap.read()
    if ret == False: break
    (kp, descs) = sift.detectAndCompute(frame, None)
    
#     descs_minmax = min_max_scaler.fit_transform([concat(descs[k], kp[k]) for k in range(0, len(kp))])
    descs_minmax = preprocessing.normalize([concat(descs[k], kp[k]) for k in range(0, len(kp))], norm='l2')
#     descs_minmax = [concat(descs[k], kp[k]) for k in range(0, len(kp))]
    lables = clf.predict(descs_minmax)
    lables
    kp_pred = []
    kp_norm = []
    for k in range(0, len(lables)):
        if lables[k] == 1:
            kp_pred.append(kp[k])
#             kp_pred.append((kp[k].pt[0], kp[k].pt[1]))
            kp_norm.append([int(kp[k].pt[0]), int(kp[k].pt[1]), int(kp[k].size / 2)])
#             cv2.circle(frame, (kp_norm[-1][0], kp_norm[-1][1]), int(kp_norm[-1][2]), (200, 20, 20), 5)
    cluster_num = 8
    if len(kp_norm) < cluster_num:
        cluster_num = len(kp_norm)
    if cluster_num > 0:
        kmeans = KMeans(n_clusters=cluster_num, random_state=0).fit(kp_norm)
        colors = [(200, 0, 0), (0, 200, 0), (0, 0, 200), (50, 100, 50), (200, 200, 200), (100, 50, 50), (50, 50, 100), (200, 0, 250)]
        for k in range(0, len(kp_norm)):
#             cv2.circle(frame, (kp_norm[k][0], kp_norm[k][1]), int(kp_norm[k][2]), colors[kmeans.labels_[k]], 5)

            cv2.circle(frame, (kp_norm[k][0], kp_norm[k][1]), 3, colors[kmeans.labels_[k]], 5)
#     else:
#         cv2.circle(frame, (kp_norm[][0], kp_norm[k][1]), int(kp_norm[k][2]), colors[kmeans.labels_[k]], 5)
    print("kp: " + str(len(kp)) + " kp_pred: " + str(len(kp_pred)))
    
#     x,y,w,h = cv2.boundingRect(kp_pred)
#     cv2.rectangle(frame, (x, y), (x + w, y + h), (220, 20, 20), 2)
#     frame = cv2.drawKeypoints(frame, kp, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

    for box in bbox:
        cv2.rectangle(frame, box[0], box[1], (20, 220, 20), 2)
    
    cv2.imshow("vid", frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()   

kp: 132 kp_pred: 47
kp: 129 kp_pred: 45
kp: 119 kp_pred: 43
kp: 130 kp_pred: 41
kp: 121 kp_pred: 39
kp: 112 kp_pred: 45
kp: 118 kp_pred: 41
kp: 109 kp_pred: 49
kp: 104 kp_pred: 50
kp: 113 kp_pred: 45
kp: 104 kp_pred: 44
kp: 86 kp_pred: 42
kp: 95 kp_pred: 50
kp: 72 kp_pred: 41
kp: 82 kp_pred: 49
kp: 84 kp_pred: 55
kp: 82 kp_pred: 54
kp: 76 kp_pred: 50
kp: 75 kp_pred: 54
kp: 74 kp_pred: 47
kp: 75 kp_pred: 54
kp: 68 kp_pred: 57
kp: 59 kp_pred: 46
kp: 41 kp_pred: 40
kp: 57 kp_pred: 49
kp: 49 kp_pred: 41
kp: 67 kp_pred: 53
kp: 70 kp_pred: 60
kp: 59 kp_pred: 49
kp: 66 kp_pred: 54
kp: 64 kp_pred: 47
kp: 60 kp_pred: 47
kp: 63 kp_pred: 48
kp: 64 kp_pred: 54
kp: 66 kp_pred: 52
kp: 69 kp_pred: 58
kp: 64 kp_pred: 52
kp: 57 kp_pred: 48
kp: 61 kp_pred: 47
kp: 62 kp_pred: 48
kp: 70 kp_pred: 54
kp: 71 kp_pred: 63
kp: 49 kp_pred: 40
kp: 40 kp_pred: 34
kp: 39 kp_pred: 30
kp: 42 kp_pred: 27
kp: 48 kp_pred: 32
kp: 38 kp_pred: 27
kp: 45 kp_pred: 25
kp: 39 kp_pred: 25
kp: 46 kp_pred: 24
kp: 49 kp_pred: 29
k

In [70]:
kmeans = KMeans(n_clusters=8, random_state=0).fit(kp_norm)
kmeans.labels_
colors = [(200, 0, 0), (0, 200, 0), (0, 0, 200), (50, 100, 50), (200, 200, 200), (100, 50, 50), (50, 50, 100), (200, 0, 250)]
for k in range(0, len(kp_norm)):
    cv2.circle(frame, (kp_norm[k][0], kp_norm[k][1]), int(kp_norm[k][2]), colors[kmeans.labels_[k]], 5)

cv2.imshow("vid", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()  

In [31]:
clusters = [[], [], [], [], [], [], [], []]
for l in range(0, 8):
    for k in range(0, len(kp_norm)):
        if kmeans.labels_[k] == l:
            clusters[l].append(np.asarray((kp_norm[k][0], kp_norm[k][1]), dtype=np.int))
    x,y,w,h = cv2.boundingRect(clusters[l])
    cv2.rectangle(frame, (x, y), (x + w, y + h), (220, 20, 20), 2)
cv2.imshow("vid", frame)
cv2.waitKey(0)
cv2.destroyAllWindows() 

TypeError: points is not a numpy array, neither a scalar